In [107]:
from multiprocessing import Process
import numpy as np
import pandas as pd
import requests
from tqdm import tqdm

import cv2
import netCDF4 as nc
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon

import json

from subprocess import run, DEVNULL

from torch.nn import Sequential, Sigmoid
from torch import from_numpy, inference_mode
from torch.nn import DataParallel

from monai.networks.nets import SwinUNETR
from monai.inferers import sliding_window_inference

from pytorch_lightning import LightningModule

from zipfile import ZipFile
from zipfile import BadZipFile
from zipfile import is_zipfile

import xmltodict

import time

import os
from os.path import join, exists, getsize, isfile

In [108]:
class DownloadSARSentinel:
    def __init__(self, email, 
             password,
             product_list,
             satellite="sentinel_1"
                ):
        self.password = password
        self.email = email
        self.product_list = product_list
        self.satellite = satellite
        self.raw_folder = None
        self._query_result = None

    def query_result(self):
        if not isinstance(self._query_result, pd.DataFrame):
            self._query_result = self.query_sar()
        return self._query_result
        
    def query_sar(self):
        API_URL_NAME = "https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=contains(Name,'S1A') and ContentDate/Start gt 2022-05-03T00:00:00.000Z and ContentDate/Start lt 2022-05-21T00:00:00.000Z"
        API_URL_NAME = "https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=contains(Name,'{name}')"
        query_result = pd.DataFrame()

        for product in self.product_list:
            json = requests.get(API_URL_NAME.format(name=product)).json()
            response_result = pd.DataFrame.from_dict(json['value'])
            
            if not response_result.empty:
                print("Found:", product)
                query_result = pd.concat([query_result, response_result])   
                time.sleep(0.5)
            else:
                print("Could not find: ", product)

        return query_result
        
    @staticmethod
    def create_folder(folder_path, original_name):
        if not folder_path:
            folder_path = join(os.getcwd(), original_name)
            if not exists(folder_path):
                os.mkdir(folder_path)

        assert exists(folder_path), f"Path not found: {folder_path}"
        return folder_path

    @staticmethod
    def is_downloaded(sar_name, folder):
        sar_path = join(folder, sar_name + ".zip")
        if exists(sar_path):
            #if getsize(sar_path) > 5E9: #ver o tamanho do arquivo para baixar arquivos incompletos
            return True
        return False

    def return_headers(self):
        token_url = 'https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token'
        headers = {'Content-Type': 'application/x-www-form-urlencoded'}
        data = {
            'grant_type': 'password',
            'username': self.email,
            'password': self.password,
            'client_id': 'cdse-public'
        }
        token_response = requests.post(token_url, headers=headers, data=data).json()
        token_url = token_response["access_token"]
        download_headers = {"Authorization": f"Bearer {token_url}"}
        return download_headers

    def download_products(self, folder, overwrite=False):
        download_url = "https://zipper.dataspace.copernicus.eu/odata/v1/Products({product_id})/$value"

        for index, product in self.query_result().iterrows():
            path_product = join(folder, product["Name"] + ".zip")               
            if not DownloadSARSentinel.is_downloaded(product["Name"], folder) or overwrite:
                session = requests.Session()
                session.headers.update(self.return_headers())
                response = session.get(download_url.format(product_id=product["Id"]), headers=self.return_headers(), stream=True)
                total_size = int(response.headers.get('Content-Length', 0))
                progress_bar = tqdm(total=total_size, unit='B', unit_scale=True, desc=f'Downloading: {product["Name"]}', leave=True)
                progress_bar.reset()

                with open(path_product, "wb") as file:
                    for chunk in response.iter_content(chunk_size=8192):
                        if chunk:
                            file.write(chunk)
                            progress_bar.update(len(chunk))
                progress_bar.reset()




        

QUERY_CSV_PATH = "/mnt/camobi_2/PHMG/Sentinel_Acquisition/New_sar_img.csv"
FILE_NAME_COLUMN = "NOME DO ARQUIVO"
RAW_SAFE_FILE = "/mnt/camobi_process/new_data/new_SAR_img"
UNZIP_SAFE_FILE = "/mnt/camobi_process/new_data/unzip_sar_img"
query_sar = pd.read_csv(QUERY_CSV_PATH, header=0)[FILE_NAME_COLUMN]

sar_list = []

for sar_name in query_sar:
    sar_path = join(UNZIP_SAFE_FILE, sar_name + ".SAFE")
    if not exists(sar_path):
        sar_list.append(sar_name)
#print(len(sar_list))

new_products = DownloadSARSentinel("pedro.meirelles@ufba.br", "Thermal1234@", product_list=sar_list)
#new_products.query_result()

#new_products.download_products(folder="/mnt/camobi_process/new_data/new_SAR_img")

In [109]:
class SentinelProduct:
    def __init__(self, name, file_path, nc_graph_path,  rect_corners=None):
        self.name = name
        self.simple_name = name.split(".")[0][-7:-4]
        self.rect_corners = rect_corners
        self.file_path = file_path
        self.nc_graph_path = nc_graph_path
        self.unzip_path = None
        self.netcdf_path = None

    def __str__(self):
        return self.file_path.split("/")[-1].split(".")[0]

    def unzip(self, unzip_folder):
        assert exists(unzip_folder), f"Folder \"{unzip_folder}\" does not exist!"
        self.unzip_path = join(unzip_folder, self.name + ".SAFE") 
        try:
            print("Unziping:", self.name)
            with ZipFile(self.file_path, 'r') as zip_ref:
                zip_ref.extractall(unzip_folder)
        except BadZipFile:
            print(f"Imposible to unzip: {self.name}. file is incomplete or corrupted!")

    def edit_zip_to_nc(self, netcdf_folder):
        with open(self.nc_graph_path) as arquivo:
            dados = xmltodict.parse(arquivo.read())

        if self.unzip_path:
            input_path = self.unzip_path
        else:
            input_path = self.file_path
        
        self.netcdf_path = join(netcdf_folder, self.simple_name + ".nc")
        print(self.netcdf_path)
        

        dados['graph']['node'][0]['parameters']['file'] = input_path 
        dados['graph']['node'][-1]['parameters']['file'] = self.netcdf_path 

        print(dados['graph']['node'][0]['parameters']['file'])


        with open(self.nc_graph_path, 'w') as arquivo:
            arquivo.write(xmltodict.unparse(dados, pretty=True))

    def convert_to_netcdf4(self, gpt_path, netcdf_folder):
        self.edit_zip_to_nc(netcdf_folder)
        shell = run([gpt_path, self.nc_graph_path])#, stdout=DEVNULL, stderr=DEVNULL)



In [110]:
class PedroNet(LightningModule): #out_channels = numero de classes
    def __init__(self, img_size, lr,
                 depths=(2, 2, 2, 2), 
                 num_heads=(3, 6, 12, 24), 
                 feature_size=24, 
                 norm_name='instance', 
                 drop_rate=0.0, 
                 attn_drop_rate=0.0, 
                 dropout_path_rate=0.0, 
                 normalize=True, 
                 use_checkpoint=False, 
                 downsample='merging', 
                 use_v2=False 
                 ):
        super().__init__()
        self.model = Sequential(SwinUNETR(spatial_dims=2,
                                    in_channels=1,
                                    out_channels=1,
                                    depths=depths,
                                    img_size=img_size,
                                    feature_size=feature_size,
                                    drop_rate=drop_rate,
                                    num_heads=num_heads,
                                    norm_name=norm_name,
                                    attn_drop_rate=attn_drop_rate,
                                    dropout_path_rate=dropout_path_rate,
                                    normalize=normalize,
                                    use_checkpoint=use_checkpoint,
                                    downsample=downsample,
                                    #use_v2=use_v2, apenas para versões mais recentes
                                    ))
    def forward(self, x):
        return self.model(x)
    
    def netcdf_inferece(self, nc_product):
        torch_img = from_numpy(np.asarray(nc_product.nc_img)).to("cuda")
        torch_img = torch_img.unsqueeze(0).unsqueeze(0)

        with inference_mode():
            model_img = sliding_window_inference(torch_img, 
                                            roi_size=(512),
                                            sw_batch_size=20, 
                                            predictor=DataParallel(self.model), 
                                            mode='constant',
                                            overlap=0.8,
                                            progress=True
                                            )

            sigmoid_fn = Sigmoid()
            model_img = sigmoid_fn(model_img)
            model_img = model_img.to("cpu").squeeze()
        return model_img

class NetcdfProduct:
    def __init__(self, product, image_variable='Sigma0_VV_db'):
        self.product = product
        self.name = product.filepath().split("/")[-1].split(".")[0]
        self.image_variable = image_variable

    @property
    def nc_img(self):
        return self.product.variables[self.image_variable][:]
    

    def polygons_to_img(self, json_path, dict_label, no_land_path=None, land_name="land"):
        with open(json_path, 'r') as f:
            json_labelme = json.load(f)

        dict_patches = {key: [] for key in dict_label.keys()}
        height = json_labelme['imageHeight']
        width = json_labelme['imageWidth']
        mask_array = np.where(self.nc_img.mask, np.nan, 0)

        for shapes in json_labelme["shapes"]:
            for number, name in dict_label.items():
                if shapes['label'] in name:
                    points = np.array(shapes['points'], dtype=int)
                    dict_patches[number].append(points)

        if no_land_path:
            land_label = list(np.load(no_land_path).values())
            for number, name in dict_label.items():
                if land_name in name:
                    dict_patches[number] = land_label

        for index, polygons in dict_patches.items():
            mask_array = cv2.fillPoly(mask_array, polygons, color=index)
        return mask_array
    
    def __str__(self):
        return self.name

    def create_img(self, folder_path):
        img_path = join(folder_path, self.name + ".png")
        normalized_img = ((self.nc_img - np.min(self.nc_img)) / (np.max(self.nc_img) - np.min(self.nc_img))) * 255
        cv2.imwrite(img_path, normalized_img)
    
    @staticmethod
    def create_polygons(mask):
        edited_contours = []
        binary_image = np.array(mask)
        
        contours, hierarchy = cv2.findContours(binary_image.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        for poly in contours:
            if len(poly) > 50:
                approx = cv2.approxPolyDP(poly, 0.8, True)
                approx = np.squeeze(approx)
                edited_contours.append(approx)
        return edited_contours
    
    @staticmethod
    def mask_to_bool(probs, threshold=0.5):
        mask = np.asarray(probs)
        mask[mask > threshold] = True
        mask[mask < threshold] = False
        return mask

    def land_mask(self, gptpath, xmlfile):
        with open(xmlfile, 'r') as f:
            dados = xmltodict.parse(f.read())

        output_file = "/mnt/camobi_2/PHMG/Sentinel_Acquisition/no_land/land.nc"

        dados['graph']['node'][0]['parameters']['file'] = self.product.filepath() 
        dados['graph']['node'][-1]['parameters']['file'] = output_file

        with open(xmlfile, 'w') as arquivo:
            arquivo.write(xmltodict.unparse(dados, pretty=True))

        shell = run([gptpath, xmlfile])#, stdout=DEVNULL, stderr=DEVNULL)
        mask_netcdf = NetcdfProduct(nc.Dataset(output_file))
        #return mask_netcdf.nc_img.mask
        mask_output = mask_netcdf.nc_img.mask
        del mask_netcdf
        os.remove(output_file)
        time.sleep(1)
        return mask_output ^ self.nc_img.mask

    def save_land_mask(self, folder, gptpath, xmlfile, simplify=2, limiar=100):
        land_mask = self.land_mask(gptpath, xmlfile)
        contours, hierarchy = cv2.findContours(land_mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        edited_contours = []
        for poly in contours:
            if len(poly) > limiar:
                approx = cv2.approxPolyDP(poly, simplify, True)
                approx = np.squeeze(approx)
                print(f"Original: {len(poly)}; Simplification: {len(approx)}" )
                edited_contours.append(approx)
        np.savez(join(folder,"no_land_" + self.name), *edited_contours)

    def patches_2D(self, size, label):
        chanel_img = np.stack([self.nc_img, label], axis=-1)
        pad_array = np.repeat(int(size/2), chanel_img.shape[-1])
        zero_pad = np.repeat(0, chanel_img.shape[-1])
        chanel_img = np.pad(chanel_img, (pad_array, pad_array, zero_pad), constant_values=np.nan)

        patches = patchify(chanel_img, (size, size, chanel_img.shape[-1]), size)
        return np.squeeze(patches)
    
    def patches_1D(self, size, label):
        patches = self.patches_2D(size, label)
        patches = patches.reshape(-1, size, size, patches.shape[-1])
        return patches

    def dict_pathes(self, size, label, dict_label, label_pos=1):
        dict_patches = {key: [] for key in dict_label.keys()}
        patches = self.patches_1D(size, label)
        for patch in patches:
            for key, list in dict_patches.items():
                label_patch = np.array(patch[:,:, label_pos], dtype="uint8")
                if np.any(label_patch == key) and np.all(~np.isnan(label_patch)): #choosing the last chanel to append in dict
                    dict_patches[key].append(patch)
                    break
                    
        return dict_patches

    def probs_to_labelme(self, probs, folder, threshold=0.5):
        mask = NetcdfProduct.mask_to_bool(probs, threshold)
        masked_polygons = NetcdfProduct.create_polygons(mask)
        labelme_format = {"version": "5.1.1",
                          "flags": {},
                          "shapes": [],
                          "imagePath": f"..\\Sar_img\\{self.name}.png",  # Update with your image filename
                          "imageData": None,
                          "imageHeight": self.product.dimensions["y"].size,
                          "imageWidth": self.product.dimensions["x"].size
                          }

        for patch in masked_polygons:
            labelme_format["shapes"].append({
                "label": "oil", 
                "points": patch.squeeze().tolist(),
                "group_id": None,
                "description": "",
                "shape_type": "polygon",
                "flags": {}
            })

        auto_labels_path = join(folder, f"{self.name}.json")
        with open(auto_labels_path, 'w') as json_file:
            json.dump(labelme_format, json_file, indent=2)
        print(f"Label \"{self.name}.json\" created!")


NETCDF_PRODUCT_PATH = "/mnt/camobi_process/new_data/images_nc"

list_nc_product = []

for nc_data in tqdm(os.listdir(NETCDF_PRODUCT_PATH)):
    nc_data_path = join(NETCDF_PRODUCT_PATH, nc_data)
    nc_img = nc.Dataset(nc_data_path, 'r')
    netcdf_sar = NetcdfProduct(nc_img)
    list_nc_product.append(netcdf_sar)
    #netcdf_sar.create_img("/mnt/camobi_2/PHMG/Sentinel_Acquisition/img_folder")

LABELS_FOLDER = "/mnt/camobi_2/PHMG/Sentinel_Acquisition/auto_labels"
WEIGHTS_MODEL = "/mnt/camobi_2/PHMG/PedroSwinNet/Model_512Img_24Feature_(2, 2, 2, 2)depths_0.0attnDrop_(3, 6, 12, 24)Heads_30.000000Lr_0drop_v1/model-Val_loss=0.003714-Val_Precision=0.973-Recall=0.971-Val_F1_Score=0.972.ckpt"
loaded_model = PedroNet.load_from_checkpoint(WEIGHTS_MODEL)


for nc_image in tqdm(list_nc_product):
    probs = loaded_model.netcdf_inferece(nc_image)
    nc_image.probs_to_labelme(probs, LABELS_FOLDER)

#plt.imshow(test_mask)
test_mask_1 = np.array(test_mask)
test_mask_1[netcdf_sar.nc_img.mask] = np.nan
#plt.imshow(test_mask_1)
netcdf_sar.probs_to_labelme(test_mask_1, LABELS_FOLDER)

In [111]:
class SentinelAcquisition:
    def __init__(self, data):
        self.column_name = "product_name"
        self.column_obg_product = "obj_product"
        self.column_product_path = "product_path"
        self.column_netcdf_path = "netcdf_path"
        self.column_obj_netcdf = "obj_netcdf"
        self.column_json_path = "json_path"
        self.column_image_path = "image_path"
        self.data = pd.DataFrame({self.column_name: data,
                                  self.column_obg_product: np.nan,
                                  self.column_product_path: np.nan,
                                  self.column_netcdf_path: np.nan,
                                  self.column_obj_netcdf: np.nan,
                                  self.column_image_path: np.nan,
                                  self.column_json_path: np.nan
                                 })

    def __str__(self):
        return self.data.to_string()

    def add_products(self, folder, nc_graph_path):
        for product in os.listdir(folder):
            if "SAFE" in product:
                product_name = product.split(".")[0]
                product_path = join(folder, product)
                search_result = self.nloc(product_name)
                if isinstance(search_result, pd.DataFrame):
                    product_obj = SentinelProduct(product_name, product_path, nc_graph_path)
                    self.data.loc[search_result.index, self.column_product_path] = product_path
                    self.data.loc[search_result.index, self.column_obg_product] = product_obj
                    
                    #display(self.data.loc[search_result.index])
                else:
                    return 

    def download_products(self, folder, email, password):
        #pending_download = self.data.loc[~self.data.index.isin(self.data.dropna(how="any").index)][self.column_name]
        subset = [self.column_product_path, self.column_netcdf_path]
        pending_download = self.data[self.data[subset].isnull().all(axis=1)][self.column_name]
        #print(pending_download)
        pending_download = list(pending_download)
        new_products = DownloadSARSentinel(email, password, product_list=pending_download)
        new_products.download_products(folder=folder)

    def add_netcdfs(self, folder, image_variable='Sigma0_VV_db'):
        for nc_product in os.listdir(folder):
            nc_product_path = join(folder, nc_product)
            nc_name = nc_product.split(".")[0]
            search_result = self.nloc(nc_name)
            if isinstance(search_result, pd.DataFrame):
                netcdf_obj = NetcdfProduct(nc.Dataset(nc_product_path, 'r'))
                self.data.loc[search_result.index, self.column_netcdf_path] = nc_product_path
                self.data.loc[search_result.index, self.column_obj_netcdf] = netcdf_obj

    def add_labels(self, folder):
        for json_label in os.listdir(folder):
            json_label_path = join(folder, json_label)
            label_name = json_label.split(".")[0]
            search_result = self.nloc(label_name)
            if isinstance(search_result, pd.DataFrame):
                self.data.loc[search_result.index, self.column_json_path] = json_label_path

    def add_image(self, folder):
        for image in os.listdir(folder):
            image_path = join(folder, image)
            image_name = image.split(".")[0]
            search_result = self.nloc(image_name)
            if isinstance(search_result, pd.DataFrame):
                self.data.loc[search_result.index, self.column_image_path] = image_path

    def set_json_path(self, folder):
        for index, row in self.data.dropna(subset=[self.column_json_path, self.column_image_path]).iterrows():
            print(row[self.column_json_path])
            json_file = json.load(open(row[self.column_json_path], "r"))
            new_path = join(folder, row[self.column_image_path].split("/")[-1])
            json_file["imagePath"] = new_path
            with open(row[self.column_json_path], 'w', encoding='utf-8') as f:
                json.dump(json_file, f, ensure_ascii=False, indent=4)

        
    def to_netcdf4(self, folder):
        pass     

    def __getitem__(self, index):
        return self.data.iloc[index]

    def nloc(self, name):
        output = self.data[self.data[self.column_name].str.contains(name)]
        if output.empty:
            print(name, "Not found!")
        else:  
            return output

In [112]:
SAR_TO_NC_GRAPH = "/mnt/camobi_2/PHMG/Sentinel_Acquisition/graphs/ZIP_to_NC.xml"
FILE_NAME_COLUMN = "NOME DO ARQUIVO"
NETCDF_FOLDER ="/mnt/camobi_3/new_data/images_nc"
data_pd = pd.read_csv("/mnt/camobi_2/PHMG/Sentinel_Acquisition/New_sar_img.csv", header=0)[FILE_NAME_COLUMN]
data_pd = list(data_pd.dropna(how="all"))
PATH_TO_GPT = "/home/camobi/snap/bin/gpt"
PRODUCT_FOLDER = "/mnt/camobi_3/new_data/new_SAR_img"
UNZIP_FOLDER = "/mnt/camobi_3/new_data/unzip_sar_img"
LABEL_PATH = "/mnt/camobi_3/new_data/labels_json"
IMAGE_PATH = "/mnt/camobi_3/new_data/png_netcdf"


FOLDER_PRODUCT = "/mnt/camobi_3/new_data/unzip_sar_img"

sar_data = SentinelAcquisition(data_pd)
sar_data.add_products("/mnt/camobi_3/new_data/unzip_sar_img", SAR_TO_NC_GRAPH)
sar_data.add_netcdfs(NETCDF_FOLDER)
sar_data.add_labels(LABEL_PATH)
sar_data.add_image(IMAGE_PATH)
#sar_data.set_json_path(r"C:\Users\goesp\Downloads\Sar_img")
#sar_data.download_products("/mnt/camobi_3/new_data/new_SAR_img")


/tmp/ipykernel_56930/3027500246.py:53: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '/mnt/camobi_3/new_data/images_nc/8578.nc' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  self.data.loc[search_result.index, self.column_netcdf_path] = nc_product_path
/tmp/ipykernel_56930/3027500246.py:54: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '8578' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  self.data.loc[search_result.index, self.column_obj_netcdf] = netcdf_obj


C8FB Not found!
A618 Not found!
2102 Not found!
32C7 Not found!
2102 Not found!
2102 Not found!
C8FB Not found!
A618 Not found!
32C7 Not found!


/tmp/ipykernel_56930/3027500246.py:62: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '/mnt/camobi_3/new_data/labels_json/CD82.json' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  self.data.loc[search_result.index, self.column_json_path] = json_label_path
/tmp/ipykernel_56930/3027500246.py:70: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '/mnt/camobi_3/new_data/png_netcdf/C184.png' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  self.data.loc[search_result.index, self.column_image_path] = image_path


In [113]:
pd.set_option('display.max_rows', 100)
pd.options.display.max_colwidth = 100
display(sar_data.data)
#sar_data.download_products()

,product_name,obj_product,product_path,netcdf_path,obj_netcdf,image_path,json_path
0,S1A_IW_SLC__1SDV_20181009T171427_20181009T171454_024062_02A131_CCBB,NaN,NaN,/mnt/camobi_3/new_data/images_nc/CCBB.nc,CCBB,/mnt/camobi_3/new_data/png_netcdf/CCBB.png,/mnt/camobi_3/new_data/labels_json/CCBB.json
1,S1B_IW_SLC__1SDV_20181008T172144_20181008T172211_013064_01822C_F065,NaN,NaN,/mnt/camobi_3/new_data/images_nc/F065.nc,F065,/mnt/camobi_3/new_data/png_netcdf/F065.png,/mnt/camobi_3/new_data/labels_json/F065.json
2,S1B_IW_SLC__1SDV_20210120T080527_20210120T080554_025235_030137_BB07,NaN,NaN,/mnt/camobi_3/new_data/images_nc/BB07.nc,BB07,NaN,/mnt/camobi_3/new_data/labels_json/BB07.json
3,S1A_IW_SLC__1SDV_20170810T024712_20170810T024738_017855_01DEF7_445E,NaN,NaN,/mnt/camobi_3/new_data/images_nc/445E.nc,445E,NaN,/mnt/camobi_3/new_data/labels_json/445E.json
4,S1A_IW_SLC__1SDV_20170729T024712_20170729T024737_017680_01D9A0_A8A3,NaN,NaN,/mnt/camobi_3/new_data/images_nc/A8A3.nc,A8A3,NaN,/mnt/camobi_3/new_data/labels_json/A8A3.json
5,S1A_IW_SLC__1SDV_20210612T235346_20210612T235415_038314_048582_23F3,NaN,NaN,NaN,NaN,NaN,/mnt/camobi_3/new_data/labels_json/23F3.json
6,S1B_IW_SLC__1SDV_20211003T014926_20211003T014953_028965_0374DA_4688,NaN,NaN,NaN,NaN,NaN,/mnt/camobi_3/new_data/labels_json/4688.json
7,S1B_IW_SLC__1SDV_20211121T142946_20211121T143013_029687_038B19_5510,NaN,NaN,NaN,NaN,NaN,/mnt/camobi_3/new_data/labels_json/5510.json
8,S1A_IW_SLC__1SDV_20210805T000201_20210805T000228_039087_049CB9_D205,NaN,NaN,NaN,NaN,NaN,/mnt/camobi_3/new_data/labels_json/D205.json
9,S1A_IW_SLC__1SDV_20170709T000134_20170709T000201_017387_01D0AA_205D,NaN,NaN,NaN,NaN,NaN,/mnt/camobi_3/new_data/labels_json/205D.json


In [114]:
sar_data.download_products("/mnt/camobi_3/new_data/new_SAR_img", "pedro.meirelles@ufba.br", "Thermal123@")

Found: S1A_IW_SLC__1SDV_20210612T235346_20210612T235415_038314_048582_23F3
Found: S1B_IW_SLC__1SDV_20211003T014926_20211003T014953_028965_0374DA_4688
Found: S1B_IW_SLC__1SDV_20211121T142946_20211121T143013_029687_038B19_5510
Found: S1A_IW_SLC__1SDV_20210805T000201_20210805T000228_039087_049CB9_D205
Found: S1A_IW_SLC__1SDV_20170709T000134_20170709T000201_017387_01D0AA_205D
Found: S1B_IW_SLC__1SDV_20190325T081332_20190325T081359_015508_01D0FD_EBAF
Found: S1A_IW_SLC__1SDV_20200203T002425_20200203T002453_031081_039239_CC57
Found: S1A_IW_SLC__1SSV_20151213T143637_20151213T143704_009025_00CF19_09E6
Found: S1A_IW_SLC__1SDV_20170717T024711_20170717T024736_017505_01D444_DB4F
Found: S1A_IW_SLC__1SDV_20170326T023848_20170326T023915_015857_01A210_CDC3
Found: S1A_IW_SLC__1SDV_20170308T142433_20170308T142459_015602_019A78_BCCF
Found: S1A_IW_SLC__1SDV_20170311T021504_20170311T021528_015638_019B8B_904B
Found: S1B_IW_SLC__1SDV_20210413T095627_20210413T095655_026447_032847_2EBC
Found: S1B_IW_SLC__1SDV_2